# Cross Matching

In [1]:
#importing libraries
import numpy as np

In [2]:
#importing the AT20G bss catalogue
def import_bss():
  cat = np.loadtxt('./catalogues/bss.dat', usecols=range(1, 7))
  my_list = []
  for i, list_ in enumerate(cat):
    ra = 15 * (list_[0] + list_[1]/60 + list_[2]/(60*60))
    dec = (list_[3]/abs(list_[3])) * (abs(list_[3]) + list_[4]/60 + list_[5]/(60*60))
    my_list.append((i + 1, ra, dec))
  return my_list

#importing the SuperCOSMOS catalogue
def import_super():
  cat = np.loadtxt('./catalogues/super.csv', delimiter=',', skiprows=1, usecols=[0, 1])
  my_list = []
  for i, list_ in enumerate(cat):
    my_list.append((i + 1, list_[0], list_[1]))
  return my_list

#calculating the angular distance over the celestial sphere of two coordinates using haversine formula
def angular_dist(ra1, dec1, ra2, dec2):
  r1 = np.radians(ra1)
  d1 = np.radians(dec1)
  r2 = np.radians(ra2)
  d2 = np.radians(dec2)
  a = np.sin(np.abs(d1 - d2)/2)**2
  b = np.cos(d1)*np.cos(d2)*np.sin(np.abs(r1 - r2)/2)**2
  d = np.degrees(2*np.arcsin(np.sqrt(a + b)))
  return d

#crossmatching two catalogues to get matches and no matches 
def crossmatch(bss_cat, super_cat, max_radius):
  matches = []
  no_matches = []
  for i in bss_cat:
    closest_dist = 180
    ra1, dec1 = i[1], i[2]
    for j in super_cat:
      ra2, dec2 = j[1], j[2]
      d = angular_dist(ra1, dec1, ra2, dec2)
      if (d < closest_dist):
        closest_dist = d
        my_tuple = (j[0], closest_dist)
    if (closest_dist > max_radius):
      no_matches.append(i[0])
    else:
      matches.append((i[0], my_tuple[0], my_tuple[1]))
  return matches, no_matches

if __name__ == '__main__':
  bss_cat = import_bss()
  super_cat = import_super()

  max_dist = 40/3600
  matches, no_matches = crossmatch(bss_cat, super_cat, max_dist)
  print(matches[:3])
  print(no_matches[:3])
  print(len(no_matches))

  max_dist = 5/3600
  matches, no_matches = crossmatch(bss_cat, super_cat, max_dist)
  print(matches[:3])
  print(no_matches[:3])
  print(len(no_matches))

[(1, 2, 0.00010988610938710059), (2, 4, 0.0007649845967242494), (3, 5, 0.00020863352870707666)]
[5, 6, 11]
9
[(1, 2, 0.00010988610938710059), (2, 4, 0.0007649845967242494), (3, 5, 0.00020863352870707666)]
[5, 6, 11]
40
